In [0]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

## Importing & Cleaning  Wildfires Dataframe

In [0]:
Historical_Wildfires_df = spark.read.format('csv').options(header='true').load('dbfs:/FileStore/Prediction_data/Historical_Wildfires.csv').toPandas()


In [0]:
Historical_Wildfires_df['Date'] = pd.to_datetime(Historical_Wildfires_df['Date'])
columns_to_convert = ["Estimated_fire_area", "Mean_estimated_fire_brightness", 
                      "Mean_estimated_fire_radiative_power", "Mean_confidence", 
                      "Std_confidence", "Var_confidence", "Count"]

Historical_Wildfires_df[columns_to_convert] = Historical_Wildfires_df[columns_to_convert].astype(float)
print(Historical_Wildfires_df.dtypes)

Region                                         object
Date                                   datetime64[ns]
Estimated_fire_area                           float64
Mean_estimated_fire_brightness                float64
Mean_estimated_fire_radiative_power           float64
Mean_confidence                               float64
Std_confidence                                float64
Var_confidence                                float64
Count                                         float64
Replaced                                       object
dtype: object


In [0]:
Historical_Wildfires_df['Year'] = Historical_Wildfires_df['Date'].dt.year
Historical_Wildfires_df['Month'] = Historical_Wildfires_df['Date'].dt.month

In [0]:
Historical_Wildfires_df.tail(5)

,Region,Date,Estimated_fire_area,Mean_estimated_fire_brightness,Mean_estimated_fire_radiative_power,Mean_confidence,Std_confidence,Var_confidence,Count,Replaced,Year,Month
26714,WA,2021-01-18,30.800000,330.909091,113.454545,86.636364,7.619353,58.054545,11.0,N,2021,1
26715,WA,2021-01-19,2.000000,305.950000,15.800000,98.500000,2.121320,4.500000,2.0,N,2021,1
26716,WA,2021-01-20,6.720000,335.137500,232.325000,94.250000,8.500000,72.250000,4.0,N,2021,1
26717,WA,2021-01-21,198.362182,326.340909,230.754546,93.363636,6.325620,40.013468,55.0,N,2021,1
26718,WA,2021-01-22,59.355333,342.960000,124.068889,96.533333,5.860500,34.345455,45.0,N,2021,1


#### Estimated fire area will be the target variable for our problem statement. 
Other columns are diagonistic data that should be deleted as these columns will be generated only after the prediction

In [0]:
Historical_Wildfires_df.drop(['Var_confidence', 'Mean_confidence', 'Std_confidence', 'Count','Replaced', 'Mean_estimated_fire_radiative_power', 'Mean_estimated_fire_brightness'], axis = 1, inplace=True)


#### Checking for Negative & Duplicate values

In [0]:
Historical_Wildfires_df['Estimated_fire_area'].fillna(0, inplace=True)

## Importing & Cleaning Weather Dataframe

In [0]:
weather_df = spark.read.format('csv').options(header='true').load('dbfs:/FileStore/Prediction_data/HistoricalWeather.csv').toPandas()

In [0]:
# Renaming Columns
weather_df = weather_df.rename(columns={"count()[unit: km^2]": "Area", "min()": "Min",
                                       "max()": "Max", "mean()": "Mean", "variance()": "Variance"})


#### Pivoting the Weather table

In [0]:
# Reformat the data
weather_df['Date'] = pd.to_datetime(weather_df['Date'])
df_pivot = weather_df.pivot_table(values=['Min','Max','Mean','Variance'], index=['Date','Region'],
                                  columns=['Parameter'])
# Reset dataframe index
df_pivot.reset_index(inplace=True)

# Renaming Column names
df_pivot.columns = [col[0] if not(col[1]) else '{1}_{0}'.format(*col) for col in df_pivot.columns.values]

# Rearranging Data and column
params = df_pivot.columns.tolist()[3:]
params.sort()
weather_data = df_pivot[df_pivot.columns.tolist()[:3] + params].copy()

In [0]:
# Creating 3 new columns - helpful while joining tables
weather_data['Year'] = weather_data['Date'].dt.year
weather_data['Month'] = weather_data['Date'].dt.month

In [0]:
# Impute missing values with 0
weather_data.fillna(0, inplace=True)

## Importing & Cleaning  Vegetation Dataframe

In [0]:
VegetationIndex_df = spark.read.format('csv').options(header='true').load('dbfs:/FileStore/Prediction_data/VegetationIndex.csv').toPandas()

In [0]:
# Fixing datatypes and renaming columns
VegetationIndex_df['Date'] = pd.to_datetime(VegetationIndex_df['Date'])

columns_to_convert = ["Vegetation_index_mean", "Vegetation_index_max", 
                      "Vegetation_index_min", "Vegetation_index_std", 
                      "Vegetation_index_variance"]

VegetationIndex_df[columns_to_convert] = VegetationIndex_df[columns_to_convert].astype(float)

In [0]:
# Creating 3 new columns - helpful while joining tables

VegetationIndex_df['Year'] = VegetationIndex_df['Date'].dt.year
VegetationIndex_df['Month'] = VegetationIndex_df['Date'].dt.month

## Importing & Cleaning Land Class Dataframe

In [0]:
LandClass_df = spark.read.format('csv').options(header='true').load('dbfs:/FileStore/Prediction_data/LandClass.csv').toPandas()

In [0]:
# Fixing data types
columns_to_convert = ['Herbaceous vegetation',
                     'Cultivated and managed vegetation/agriculture (cropland)',
                     'Urban / built up',
                     'Bare / sparse vegetation',
                     'Permanent water bodies',
                     'Herbaceous wetland',
                     'Closed forest, evergreen, broad leaf',
                     'Closed forest, deciduous broad leaf',
                     'Closed forest, unknown',
                     'Open forest, evergreen broad leaf',
                     'Open forest, deciduous broad leaf',
                     'Open forest, unknown definitions',
                     'Open sea']

LandClass_df[columns_to_convert] = LandClass_df[columns_to_convert].astype(float)

# Exploratory Data Analysis

## Wildfire Dataframe

### Visualizing Estimated_fire_area over the years

#### Fixing Ouliers - Imputing values higher than 3000 with 3000

In [0]:
Historical_Wildfires_df.loc[Historical_Wildfires_df['Estimated_fire_area'] > 3000, 'Estimated_fire_area'] = 3000

### Visualizing Estimated_fire_area over the Months

### Visualizing fire area for a Year

## Weather Dataframe

## Vegetation Dataframe

#### Checking for Missing values

### Vegetation Index by Region

# Feature Engineering

## Correlations
### Landclass Dataframe

In [0]:
# Dropping columns - Multicollinearity
LandClass_data = LandClass_df.drop(['Urban / built up', 'Open forest, evergreen broad leaf','Herbaceous vegetation', 'Open sea', 'Closed forest, evergreen, broad leaf', 'Herbaceous wetland','Open forest, deciduous broad leaf','Open forest, unknown definitions','Closed forest, unknown' ], axis = 1)

### Weather Dataframe

In [0]:
# Dropping columns - Multicollinearity
weather_df1 = weather_data.drop(['Precipitation_Variance', 'RelativeHumidity_Variance', 'SoilWaterContent_Variance', 'SolarRadiation_Variance','SolarRadiation_Min', 'Temperature_Variance', 'WindSpeed_Variance', 'RelativeHumidity_Min', 'RelativeHumidity_Max', 'SolarRadiation_Max', 'Temperature_Min', 'Temperature_Max', 'SoilWaterContent_Mean', 'WindSpeed_Min'], axis=1)

### Vegetation Dataframe

In [0]:
# Dropping due to Multicollinearity
VegetationIndex_df.drop(['Vegetation_index_variance'], inplace=True, axis=1)

# Joining the tables

In [0]:
df_all1 = weather_df1.join(LandClass_data.set_index('Region'), on='Region', how='left')

In [0]:
df_all2 = pd.merge(df_all1, VegetationIndex_df, on=['Region', 'Year', 'Month'], how='left')

In [0]:
df_all2.drop(['Date_y'], axis=1,inplace=True )
df_all2.rename(columns={'Date_x': 'Date'}, inplace=True)

In [0]:
df_all = pd.merge(df_all2, Historical_Wildfires_df, on=['Region', 'Date'], how='left')

In [0]:
df_all.drop(['Year_y', 'Month_y'], axis = 1, inplace=True)
df_all.rename(columns={'Year_x': 'Year', 'Month_x': 'Month'}, inplace=True)

In [0]:
# Check for missing values in df_all
df_all.isnull().sum()

Date                                                            0
Region                                                          0
Precipitation_Max                                               0
Precipitation_Mean                                              0
Precipitation_Min                                               0
RelativeHumidity_Mean                                           0
SoilWaterContent_Max                                            0
SoilWaterContent_Min                                            0
SolarRadiation_Mean                                             0
Temperature_Mean                                                0
WindSpeed_Max                                                   0
WindSpeed_Mean                                                  0
Year                                                            0
Month                                                           0
Shrubs                                                          0
Cultivated

The missing values indicate that there were no forest fires during those dates. So imputing the area with o

In [0]:
df_all['Estimated_fire_area'].fillna(0, inplace=True)

### Checking for Correlation in the final Dataset

## Normalizing the Dataset

In [0]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder


### LabelEncoding

In [0]:
label_encoder = LabelEncoder()

# Fit and transform both 'Region' and 'Shrubs' columns
df_all['Region'] = label_encoder.fit_transform(df_all['Region'])

# Now, you can generate label mappings
region_label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))


print("Region Label Mapping:", region_label_mapping)

Region Label Mapping: {'NSW': 0, 'NT': 1, 'QL': 2, 'SA': 3, 'TA': 4, 'VI': 5, 'WA': 6}


In [0]:
df_all.columns = df_all.columns.str.replace('[,;{}()\n\t= ]', '_', regex=True)

In [0]:
df_all

,Date,Region,Precipitation_Max,Precipitation_Mean,Precipitation_Min,RelativeHumidity_Mean,SoilWaterContent_Max,SoilWaterContent_Min,SolarRadiation_Mean,Temperature_Mean,WindSpeed_Max,WindSpeed_Mean,Year,Month,Shrubs,Cultivated_and_managed_vegetation/agriculture__cropland_,Bare_/_sparse_vegetation,Permanent_water_bodies,Closed_forest__deciduous_broad_leaf,Vegetation_index_mean,Vegetation_index_max,Vegetation_index_min,Vegetation_index_std,Estimated_fire_area
0,2005-01-01,0,1.836935,0.044274,0.000000,36.355567,0.414305,0.002245,26.749389,27.341182,7.670482,3.323550,2005,1,6.2,13.0,0.2,0.2,6.8,0.349202,0.9972,-0.2,0.204862,0.000000
1,2005-01-01,1,315.266815,9.884958,0.000000,61.494675,0.496140,0.000000,19.781791,29.881492,9.704402,5.296892,2005,1,18.1,0.1,0.4,0.1,7.7,0.300478,0.9986,-0.2,0.152621,4.945000
2,2005-01-01,2,74.452164,1.453053,0.000000,47.959364,0.472416,0.000000,27.056979,28.842866,7.675632,3.483753,2005,1,9.5,1.6,1.1,0.1,13.3,0.357081,0.9995,-0.2,0.181695,38.831579
3,2005-01-01,3,3.193624,0.059078,0.000000,30.057683,0.263911,0.000000,27.142643,30.793675,10.044715,4.657538,2005,1,24.1,5.8,4.8,1.2,1.3,0.179208,0.9634,-0.2,0.076393,0.000000
4,2005-01-01,4,13.604791,3.099497,0.003973,65.086764,0.368189,0.000000,26.755711,11.788805,11.432408,5.408138,2005,1,0.7,1.2,0.1,1.9,0.6,0.638585,0.9920,-0.2,0.169840,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41008,2021-01-18,2,123.589638,5.015143,0.000000,56.017548,0.482571,0.000000,23.719916,27.882091,7.001957,3.194751,2021,1,9.5,1.6,1.1,0.1,13.3,NaN,NaN,NaN,NaN,4.000000
41009,2021-01-18,3,0.641580,0.002903,0.000000,41.457791,0.276328,0.000000,31.064747,23.793725,8.083969,4.976130,2021,1,24.1,5.8,4.8,1.2,1.3,NaN,NaN,NaN,NaN,1.430000
41010,2021-01-18,4,5.923707,1.386855,0.000000,63.776395,0.413248,0.000000,19.137688,11.530871,17.569241,7.163571,2021,1,0.7,1.2,0.1,1.9,0.6,NaN,NaN,NaN,NaN,0.000000
41011,2021-01-18,5,2.398288,0.120829,0.000000,60.200831,0.393214,0.000000,28.730524,17.403762,13.024096,5.066601,2021,1,1.4,23.3,0.1,0.6,3.8,NaN,NaN,NaN,NaN,1.000000


In [0]:
fullpredictiondata = df_all[df_all['Date'] > '2020-10-31 00:00:00']

In [0]:
fullpredictiondata.drop(['Estimated_fire_area', 'RelativeHumidity_Mean'], axis = 1, inplace=True)


/root/.ipykernel/44817/command-2453183741896277-3641393684:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fullpredictiondata.drop(['Estimated_fire_area'], axis = 1, inplace=True)


In [0]:
# Generating 7 different dataframes based on the values of Region
NSW_predictiondata = fullpredictiondata[fullpredictiondata['Region'] == 0]
NT_predictiondata = fullpredictiondata[fullpredictiondata['Region'] == 1]
QL_predictiondata = fullpredictiondata[fullpredictiondata['Region'] == 2]
SA_predictiondata = fullpredictiondata[fullpredictiondata['Region'] == 3]
TA_predictiondata = fullpredictiondata[fullpredictiondata['Region'] == 4]
VI_predictiondata = fullpredictiondata[fullpredictiondata['Region'] == 5]
WA_predictiondata = fullpredictiondata[fullpredictiondata['Region'] == 6]

In [0]:
spark.sql("CREATE DATABASE IF NOT EXISTS prediction_aus_wildfires")

DataFrame[]

In [0]:
# CONVERTING IN ON A SPARK DATAFRAME

df_all_spark = spark.createDataFrame(fullpredictiondata)
NSW_spark = spark.createDataFrame(NSW_predictiondata)
NT_spark = spark.createDataFrame(NT_predictiondata)
QL_spark = spark.createDataFrame(QL_predictiondata)
SA_spark = spark.createDataFrame(SA_predictiondata)
TA_spark = spark.createDataFrame(TA_predictiondata)
VI_spark = spark.createDataFrame(VI_predictiondata)
WA_spark = spark.createDataFrame(WA_predictiondata)

In [0]:
spark.sql("DROP TABLE IF EXISTS prediction_aus_wildfires.allregions")
# Create a temporary view from label_spark_df
df_all_spark.createOrReplaceTempView("temp_view")
# Convert the temporary view to a table
spark.sql("CREATE TABLE prediction_aus_wildfires.allregions AS SELECT * FROM temp_view")

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
spark.sql("DROP TABLE IF EXISTS prediction_aus_wildfires.NSW_prediction_data")
NSW_spark.createOrReplaceTempView("temp_view1")
spark.sql("CREATE TABLE prediction_aus_wildfires.NSW_prediction_data AS SELECT * FROM temp_view1")

spark.sql("DROP TABLE IF EXISTS prediction_aus_wildfires.NT_prediction_data")
NT_spark.createOrReplaceTempView("temp_view2")
spark.sql("CREATE TABLE prediction_aus_wildfires.NT_prediction_data AS SELECT * FROM temp_view2")

spark.sql("DROP TABLE IF EXISTS prediction_aus_wildfires.QL_prediction_data")
QL_spark.createOrReplaceTempView("temp_view3")
spark.sql("CREATE TABLE prediction_aus_wildfires.QL_prediction_data AS SELECT * FROM temp_view3")

spark.sql("DROP TABLE IF EXISTS prediction_aus_wildfires.SA_prediction_data")
SA_spark.createOrReplaceTempView("temp_view4")
spark.sql("CREATE TABLE prediction_aus_wildfires.SA_prediction_data AS SELECT * FROM temp_view4")

spark.sql("DROP TABLE IF EXISTS prediction_aus_wildfires.TA_prediction_data")
TA_spark.createOrReplaceTempView("temp_view5")
spark.sql("CREATE TABLE prediction_aus_wildfires.TA_prediction_data AS SELECT * FROM temp_view5")

spark.sql("DROP TABLE IF EXISTS prediction_aus_wildfires.VI_prediction_data")
VI_spark.createOrReplaceTempView("temp_view6")
spark.sql("CREATE TABLE prediction_aus_wildfires.VI_prediction_data AS SELECT * FROM temp_view6")

spark.sql("DROP TABLE IF EXISTS prediction_aus_wildfires.WA_prediction_data")
WA_spark.createOrReplaceTempView("temp_view7")
spark.sql("CREATE TABLE prediction_aus_wildfires.WA_prediction_data AS SELECT * FROM temp_view7")

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
%sql
select * from  prediction_aus_wildfires.NSW_prediction_data

Date,Region,Precipitation_Max,Precipitation_Mean,Precipitation_Min,RelativeHumidity_Mean,SoilWaterContent_Max,SoilWaterContent_Min,SolarRadiation_Mean,Temperature_Mean,WindSpeed_Max,WindSpeed_Mean,Year,Month,Shrubs,Cultivated_and_managed_vegetation/agriculture__cropland_,Bare_/_sparse_vegetation,Permanent_water_bodies,Closed_forest__deciduous_broad_leaf,Vegetation_index_mean,Vegetation_index_max,Vegetation_index_min,Vegetation_index_std
2020-11-01T00:00:00Z,0,13.27205753326416,0.3399253977277619,0.0,58.22523365549257,0.463790744543076,0.0040077813901009994,24.989830709534775,19.2235199334174,10.093379020690918,3.406462699654476,2020,11,6.2,13.0,0.2,0.2,6.8,0.39018727,0.966899991,-0.199999988,0.2014139
2020-11-02T00:00:00Z,0,1.670286178588867,0.046344826218342995,0.0,51.444950218985745,0.444154173135757,0.006485901772976,27.939121257593268,20.428052061234226,7.767242908477782,3.1999174185259336,2020,11,6.2,13.0,0.2,0.2,6.8,0.39018727,0.966899991,-0.199999988,0.2014139
2020-11-03T00:00:00Z,0,1.6452790498733518,0.007782246540582,0.0,45.605731771346846,0.4274033010005949,0.009144559502602,28.94655776390912,21.908099112392197,7.9098405838012695,2.806550127339705,2020,11,6.2,13.0,0.2,0.2,6.8,0.39018727,0.966899991,-0.199999988,0.2014139
2020-11-04T00:00:00Z,0,29.678232192993164,3.0130910880032764,0.0,52.197555011148616,0.412576913833618,0.0106470156461,24.257280923532964,21.208240703887018,9.172279357910156,4.529827541838683,2020,11,6.2,13.0,0.2,0.2,6.8,0.39018727,0.966899991,-0.199999988,0.2014139
2020-11-05T00:00:00Z,0,33.587207794189446,2.2636986318593904,0.0,54.10890649163142,0.45890042185783403,0.011449042707681998,25.596029143612,14.972721808661802,10.640344619750977,4.751309602890596,2020,11,6.2,13.0,0.2,0.2,6.8,0.39018727,0.966899991,-0.199999988,0.2014139
2020-11-06T00:00:00Z,0,3.222622394561768,0.050766424114472,0.0,50.58870471182375,0.43619689345359797,0.010883253067732,28.00257503272754,16.873234188073674,9.485118865966797,3.371304748029274,2020,11,6.2,13.0,0.2,0.2,6.8,0.39018727,0.966899991,-0.199999988,0.2014139
2020-11-07T00:00:00Z,0,2.9123418331146236,0.11360939808612,0.0,47.57083337331669,0.41787841916084295,0.008597824722528,28.350017717434557,18.324019686920696,6.764839172363281,4.048133728666039,2020,11,6.2,13.0,0.2,0.2,6.8,0.39018727,0.966899991,-0.199999988,0.2014139
2020-11-08T00:00:00Z,0,1.060343265533447,0.031345508465952,0.0,47.116303651870965,0.403036117553711,0.006520930211991001,27.99228816468525,20.30107241110976,6.571534156799316,3.8686170221672613,2020,11,6.2,13.0,0.2,0.2,6.8,0.39018727,0.966899991,-0.199999988,0.2014139
2020-11-09T00:00:00Z,0,1.5815242528915412,0.048110594707656,0.0,46.91309277602192,0.389001727104187,0.005316022317857,27.376894369173737,21.92566841840948,7.831758975982666,3.6036956902628887,2020,11,6.2,13.0,0.2,0.2,6.8,0.39018727,0.966899991,-0.199999988,0.2014139
2020-11-10T00:00:00Z,0,1.494732975959778,0.05358677385336401,0.0,41.6356633806419,0.37637731432914706,0.0052936999127270006,28.169132911547432,24.194212822688797,9.174653053283693,3.967836515599783,2020,11,6.2,13.0,0.2,0.2,6.8,0.39018727,0.966899991,-0.199999988,0.2014139
